<a href="https://colab.research.google.com/github/AokiMasataka/LSTM_sample/blob/master/LSTM_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U torch torchvision

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.6.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.7.0+cu101)


In [46]:
import torch
from torch import nn
import numpy as np

VOCAB_SIZE = 64
EMBEDDING_DIM = 8
sequence = 5

class Decoder(nn.Module):
    def __init__(self, hidden_dim=64):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.lstm = nn.LSTM(input_size=EMBEDDING_DIM, hidden_size=hidden_dim, batch_first=True, num_layers=1)
        self.liner = nn.Linear(hidden_dim, VOCAB_SIZE)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        _, (h, c) = self.lstm(embeds)
        output = self.liner(h[0])
        return output

In [47]:
def getData():
  numbers = []
  num = np.random.randint(0, VOCAB_SIZE - sequence)
  for i in range(sequence):
    numbers.append(num + i)
  return torch.tensor(numbers)

In [40]:
decoder = Decoder()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(decoder.parameters(), lr=0.01, momentum=0.9)

# 学習
decoder.train()
for i in range(1000):
  data = getData()
  x = data[:-1].reshape((1, -1))
  y = data[-1].reshape(1)
  h = decoder(x)
  loss = criterion(h, y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if(i + 1) % 100 == 0:
    print("epoch :", loss.item())


epoch : 3.678983688354492
epoch : 3.9468894004821777
epoch : 3.368166923522949
epoch : 4.024415969848633
epoch : 3.798377752304077
epoch : 3.547945737838745
epoch : 1.9713499546051025
epoch : 2.810211420059204
epoch : 0.3674304187297821
epoch : 1.6003410816192627


In [44]:
# Test
decoder.eval()
start = [[3]]
test_tensor = torch.tensor(start)

for i in range(sequence):
  h = decoder(test_tensor)
  number = torch.argmax(h).reshape(1)
  test_tensor = torch.cat((test_tensor, number), -1)
  print(test_tensor)

SyntaxError: ignored